# Day 15 - Deepseek R1 Lite

In [1]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    reading_map = True
    for line in lines:
        if reading_map:
            if line.startswith('#') and line.endswith('#'):
                map_lines.append(line)
            else:
                reading_map = False
                moves.append(line)
        else:
            moves.append(line)
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Parse the map to get initial positions
    walls = set()
    boxes = set()
    for row, line in enumerate(map_lines):
        for col, char in enumerate(line):
            if char == '#':
                walls.add((row, col))
            elif char == 'O':
                boxes.add((row, col))
            elif char == '@':
                robot_pos = (row, col)
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box = (start_r + dr, start_c + dc)
        while True:
            if next_box in walls_set:
                return None
            if next_box in boxes_set:
                boxes_to_push.append(next_box)
                next_box = (next_box[0] + dr, next_box[1] + dc)
            else:
                if next_box in walls_set or next_box in boxes_set:
                    return None
                pushed = {}
                for box in boxes_to_push:
                    new_r, new_c = box[0] + dr, box[1] + dc
                    pushed[box] = (new_r, new_c)
                return pushed
            if not (0 <= next_box[0] < len(map_lines) and 0 <= next_box[1] < len(map_lines[0])):
                return None
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        if (new_r, new_c) in boxes:
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                robot_pos = (new_r, new_c)
                boxes = boxes - set(pushed.keys()) | set(pushed.values())
        elif (new_r, new_c) not in walls and (new_r, new_c) not in boxes:
            robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

1436690


## Part 2

In [2]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    for line in lines:
        if line.startswith('#') and line.endswith('#'):
            map_lines.append(line)
        else:
            moves.append(line)
    
    # Scale the map
    scaled_map = []
    for line in map_lines:
        scaled_row = ''
        for char in line:
            if char == '#':
                scaled_row += '##'
            elif char == 'O':
                scaled_row += '[]'
            elif char == '.':
                scaled_row += '..'
            elif char == '@':
                scaled_row += '@.'
            else:
                scaled_row += char  # Assuming no other characters are present
        scaled_map.append(scaled_row)
    
    # Find robot and boxes positions
    robot_pos = None
    boxes = set()
    for row_idx, row in enumerate(scaled_map):
        if '@' in row:
            c = row.find('@')
            robot_pos = (row_idx, c)
        # Find all boxes '[]'
        start = 0
        while start < len(row)-1:
            if row[start:start+2] == '[]':
                boxes.add((row_idx, start))
                start += 2
            else:
                start += 1
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box_start = (start_r + dr, start_c + dc)
        # Determine how much the box moves
        box_width = 2 if dc != 0 else 1  # Horizontal boxes are 2 wide
        move_distance = 2 if dc != 0 else 1
        while True:
            # Check if next position is wall or another box
            if (next_box_start[0], next_box_start[1]) in walls_set:
                return None
            if (next_box_start[0], next_box_start[1]) in boxes_set:
                boxes_to_push.append(next_box_start)
                next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
            else:
                # Check if the space is free
                if dc == 0:
                    # Vertical push, check single column
                    if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]) in boxes_set:
                        return None
                else:
                    # Horizontal push, check two columns
                    if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]+1) in walls_set:
                        return None
                    if (next_box_start[0], next_box_start[1]) in boxes_set or (next_box_start[0], next_box_start[1]+1) in boxes_set:
                        return None
                # If all boxes can be pushed, return the new positions
                pushed = {}
                for box in boxes_to_push:
                    new_r, new_c = box[0] + dr, box[1] + dc
                    pushed[box] = (new_r, new_c)
                return pushed
            # Check boundaries
            if not (0 <= next_box_start[0] < len(scaled_map) and 0 <= next_box_start[1] < len(scaled_map[0])):
                return None
    
    # Create walls set
    walls = set()
    for row_idx, row in enumerate(scaled_map):
        for c in range(len(row)):
            if row[c] == '#':
                walls.add((row_idx, c))
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        # Determine new robot position
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        # Check if new position is a wall
        if (new_r, new_c) in walls:
            continue
        # Check if new position is a box
        if (new_r, new_c) in boxes:
            # Attempt to push the box
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                # Update robot position
                robot_pos = (new_r, new_c)
                # Update boxes positions
                for old_pos, new_pos in pushed.items():
                    boxes.remove(old_pos)
                    boxes.add(new_pos)
        # If new position is floor
        else:
            # Check if moving into floor is possible
            if dc != 0:
                # For horizontal movement, ensure both columns are free
                if new_c + 1 >= len(scaled_map[0]):
                    continue
                if (new_r, new_c) in walls or (new_r, new_c + 1) in walls:
                    continue
                if (new_r, new_c) in boxes or (new_r, new_c + 1) in boxes:
                    # Attempt to push the box
                    pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
                    if pushed is not None:
                        robot_pos = (new_r, new_c)
                        for old_pos, new_pos in pushed.items():
                            boxes.remove(old_pos)
                            boxes.add(new_pos)
                else:
                    robot_pos = (new_r, new_c)
            else:
                # Vertical movement, single column
                if (new_r, new_c) in walls:
                    continue
                if (new_r, new_c) in boxes:
                    pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
                    if pushed is not None:
                        robot_pos = (new_r, new_c)
                        for old_pos, new_pos in pushed.items():
                            boxes.remove(old_pos)
                            boxes.add(new_pos)
                else:
                    robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

KeyError: (20, 49)

In [3]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    reading_map = True
    for line in lines:
        if reading_map:
            if line.startswith('#') and line.endswith('#'):
                map_lines.append(line)
            else:
                reading_map = False
                moves.append(line)
        else:
            moves.append(line)
    
    # Scale the map
    scaled_map = []
    for line in map_lines:
        scaled_row = ''
        for char in line:
            if char == '#':
                scaled_row += '##'
            elif char == 'O':
                scaled_row += '[]'
            elif char == '.':
                scaled_row += '..'
            elif char == '@':
                scaled_row += '@.'
            else:
                scaled_row += char  # Assuming no other characters are present
        scaled_map.append(scaled_row)
    
    # Find robot and boxes positions
    robot_pos = None
    boxes = set()
    for row_idx, row in enumerate(scaled_map):
        if '@' in row:
            c = row.find('@')
            robot_pos = (row_idx, c)
        # Find all boxes '[]'
        start = 0
        while start < len(row)-1:
            if row[start:start+2] == '[]':
                boxes.add((row_idx, start))
                start += 2
            else:
                start += 1
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box_start = (start_r + dr, start_c + dc)
        while True:
            if dc != 0:
                if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]+1) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set or (next_box_start[0], next_box_start[1]+1) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            else:
                if (next_box_start[0], next_box_start[1]) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            if not (0 <= next_box_start[0] < len(scaled_map) and 0 <= next_box_start[1] < len(scaled_map[0])):
                return None
    
    # Create walls set
    walls = set()
    for row_idx, row in enumerate(scaled_map):
        for c in range(len(row)):
            if row[c] == '#':
                walls.add((row_idx, c))
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        if (new_r, new_c) in boxes:
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                robot_pos = (new_r, new_c)
                for old_pos, new_pos in pushed.items():
                    boxes.remove(old_pos)
                    boxes.add(new_pos)
        elif (new_r, new_c) not in walls and (new_r, new_c) not in boxes:
            robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

928694


In [4]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    reading_map = True
    for line in lines:
        if reading_map:
            if line.startswith('#') and line.endswith('#'):
                map_lines.append(line)
            else:
                reading_map = False
                moves.append(line)
        else:
            moves.append(line)
    
    # Scale the map
    scaled_map = []
    for line in map_lines:
        scaled_row = ''
        for char in line:
            if char == '#':
                scaled_row += '##'
            elif char == 'O':
                scaled_row += '[]'
            elif char == '.':
                scaled_row += '..'
            elif char == '@':
                scaled_row += '@.'
            else:
                scaled_row += char  # Assuming no other characters are present
        scaled_map.append(scaled_row)
    
    # Find robot and boxes positions
    robot_pos = None
    boxes = set()
    for row_idx, row in enumerate(scaled_map):
        if '@' in row:
            c = row.find('@')
            robot_pos = (row_idx, c)
        # Find all boxes '[]'
        start = 0
        while start < len(row)-1:
            if row[start:start+2] == '[]':
                boxes.add((row_idx, start))
                start += 2
            else:
                start += 1
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box_start = (start_r + dr, start_c + dc)
        while True:
            if dc != 0:
                if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]+1) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set or (next_box_start[0], next_box_start[1]+1) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            else:
                if (next_box_start[0], next_box_start[1]) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            if not (0 <= next_box_start[0] < len(scaled_map) and 0 <= next_box_start[1] < len(scaled_map[0])):
                return None
    
    # Create walls set
    walls = set()
    for row_idx, row in enumerate(scaled_map):
        for c in range(len(row)):
            if row[c] == '#':
                walls.add((row_idx, c))
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        if (new_r, new_c) in boxes:
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                robot_pos = (new_r, new_c)
                for old_pos, new_pos in pushed.items():
                    boxes.remove(old_pos)
                    boxes.add(new_pos)
        elif (new_r, new_c) not in walls and (new_r, new_c) not in boxes:
            robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

928694


In [5]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    reading_map = True
    for line in lines:
        if reading_map:
            if line.startswith('#') and line.endswith('#'):
                map_lines.append(line)
            else:
                reading_map = False
                moves.append(line)
        else:
            moves.append(line)
    
    # Scale the map
    scaled_map = []
    for line in map_lines:
        scaled_row = ''
        for char in line:
            if char == '#':
                scaled_row += '##'
            elif char == 'O':
                scaled_row += '[]'
            elif char == '.':
                scaled_row += '..'
            elif char == '@':
                scaled_row += '@.'
            else:
                scaled_row += char  # Assuming no other characters are present
        scaled_map.append(scaled_row)
    
    # Find robot and boxes positions
    robot_pos = None
    boxes = set()
    for row_idx, row in enumerate(scaled_map):
        if '@' in row:
            c = row.find('@')
            robot_pos = (row_idx, c)
        # Find all boxes '[]'
        start = 0
        while start < len(row)-1:
            if row[start:start+2] == '[]':
                boxes.add((row_idx, start))
                start += 2
            else:
                start += 1
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box_start = (start_r + dr, start_c + dc)
        while True:
            if dc != 0:
                if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]+1) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set or (next_box_start[0], next_box_start[1]+1) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            else:
                if (next_box_start[0], next_box_start[1]) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            if not (0 <= next_box_start[0] < len(scaled_map) and 0 <= next_box_start[1] < len(scaled_map[0])):
                return None
    
    # Create walls set
    walls = set()
    for row_idx, row in enumerate(scaled_map):
        for c in range(len(row)):
            if row[c] == '#':
                walls.add((row_idx, c))
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        if (new_r, new_c) in boxes:
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                robot_pos = (new_r, new_c)
                for old_pos, new_pos in pushed.items():
                    boxes.remove(old_pos)
                    boxes.add(new_pos)
        elif (new_r, new_c) not in walls and (new_r, new_c) not in boxes:
            robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

928694


In [6]:
def main():
    import sys

    # Read all lines from input file
    with open('input.txt', 'r') as f:
        lines = [line.rstrip('\n') for line in f]

    # Separate map and move sequence
    map_lines = []
    moves = []
    reading_map = True
    for line in lines:
        if reading_map:
            if line.startswith('#') and line.endswith('#'):
                map_lines.append(line)
            else:
                reading_map = False
                moves.append(line)
        else:
            moves.append(line)
    
    # Scale the map
    scaled_map = []
    for line in map_lines:
        scaled_row = ''
        for char in line:
            if char == '#':
                scaled_row += '##'
            elif char == 'O':
                scaled_row += '[]'
            elif char == '.':
                scaled_row += '..'
            elif char == '@':
                scaled_row += '@.'
            else:
                scaled_row += char  # Assuming no other characters are present
        scaled_map.append(scaled_row)
    
    # Find robot and boxes positions
    robot_pos = None
    boxes = set()
    for row_idx, row in enumerate(scaled_map):
        if '@' in row:
            c = row.find('@')
            robot_pos = (row_idx, c)
        # Find all boxes '[]'
        start = 0
        while start < len(row)-1:
            if row[start:start+2] == '[]':
                boxes.add((row_idx, start))
                start += 2
            else:
                start += 1
    
    # Define directions
    directions = {
        '^': (-1, 0),
        'v': (1, 0),
        '<': (0, -1),
        '>': (0, 1)
    }
    
    # Function to attempt to push boxes
    def push_boxes(start_r, start_c, boxes_set, walls_set, dr, dc):
        boxes_to_push = [(start_r, start_c)]
        next_box_start = (start_r + dr, start_c + dc)
        while True:
            if dc != 0:
                if (next_box_start[0], next_box_start[1]) in walls_set or (next_box_start[0], next_box_start[1]+1) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set or (next_box_start[0], next_box_start[1]+1) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            else:
                if (next_box_start[0], next_box_start[1]) in walls_set:
                    return None
                if (next_box_start[0], next_box_start[1]) in boxes_set:
                    boxes_to_push.append(next_box_start)
                    next_box_start = (next_box_start[0] + dr, next_box_start[1] + dc)
                else:
                    pushed = {}
                    for box in boxes_to_push:
                        new_r, new_c = box[0] + dr, box[1] + dc
                        pushed[box] = (new_r, new_c)
                    return pushed
            if not (0 <= next_box_start[0] < len(scaled_map) and 0 <= next_box_start[1] < len(scaled_map[0])):
                return None
    
    # Create walls set
    walls = set()
    for row_idx, row in enumerate(scaled_map):
        for c in range(len(row)):
            if row[c] == '#':
                walls.add((row_idx, c))
    
    # Concatenate move sequence into a single string
    move_sequence = ''.join(moves)
    
    # Process each move in the sequence
    for move in move_sequence:
        dr, dc = directions.get(move, (0, 0))
        new_r, new_c = robot_pos[0] + dr, robot_pos[1] + dc
        if (new_r, new_c) in boxes:
            pushed = push_boxes(new_r, new_c, boxes, walls, dr, dc)
            if pushed is not None:
                robot_pos = (new_r, new_c)
                for old_pos, new_pos in pushed.items():
                    boxes.remove(old_pos)
                    boxes.add(new_pos)
        elif (new_r, new_c) not in walls and (new_r, new_c) not in boxes:
            robot_pos = (new_r, new_c)
    
    # Calculate the sum of GPS coordinates
    sum_gps = 0
    for box in boxes:
        row, col = box
        sum_gps += 100 * row + col
    
    # Output the result
    print(sum_gps)

if __name__ == "__main__":
    main()

928694
